In [1]:
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
# import hvplot.dask
import numpy as np
import dask.array as da
import pickle
import seaborn as sns

Additional features

- base rate booking -> # booked / # found
- best hotel for location
- aggregate competitor information
  - better or equal vs. worse
  - drop non-percentage difference
- expected position
- reduce children to binary flag?

- do factor analysis?


###### stuff that were important features for others

 09 : hotel's historical price - current price
data["ump"] = exp(prop_log_historical_price) - price_usd
12 :
data["score2ma"] = prop_location_score2 * srch_query_affinity_score
 13 :
data["score1d2"] = (prop_location_score2 + 0.0001)/(prop_location_score1 + 0.0001)

data["fee_per_person"] = (price_usd * srch_room_count)/(srch_adults_count + srch_children_count)

- categorical to one-hot encoding

In [105]:
df = pd.read_csv("data/training_set_VU_DM.csv")

In [109]:
# 1. Base rate booking column
grouped_hotels = df.groupby("prop_id").sum(numeric_only=True)
temp = pd.DataFrame()
temp["prob_clickbook"] = grouped_hotels["booking_bool"]/ grouped_hotels["click_bool"]
temp.fillna(0,inplace=True)
temp =temp.reset_index()
df = df.merge(temp, on="prop_id")
del temp

,prop_id,prob_clickbook
0,1,0.0
1,2,1.0
2,3,1.0
3,4,1.0
4,5,0.0
...,...,...
129108,140817,0.0
129109,140818,0.0
129110,140819,0.0
129111,140820,0.0


In [117]:
# 2. Percentile rank of hotel in its country
grouped_hotels = df.groupby(["prop_id","prop_country_id"]).mean(numeric_only=True).reset_index()
temp = grouped_hotels[["prop_starrating", "prop_review_score","prop_id","prop_country_id"]]
temp["total_score"] = temp.loc[:,"prop_starrating"] + temp.loc[:,"prop_review_score"]
temp["country_pct_rank"] = temp.groupby("prop_country_id")["total_score"].rank(method="dense", ascending=True,pct=True)
temp = temp[["prop_id","country_pct_rank"]]
df = df.merge(temp, on="prop_id")

/var/folders/z9/j9843dsj2t5dsl2bv81xcwww0000gn/T/ipykernel_30801/574121879.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["total_score"] = temp.loc[:,"prop_starrating"] + temp.loc[:,"prop_review_score"]
/var/folders/z9/j9843dsj2t5dsl2bv81xcwww0000gn/T/ipykernel_30801/574121879.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["country_pct_rank"] = temp.groupby("prop_country_id")["total_score"].rank(method="dense", ascending=True,pct=True)


In [108]:
df.columns

Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate'

In [116]:
df

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,prob_clickbook,country_pct_rank
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0.625,0.65
1,498,2012-12-23 11:32:22,32,220,NaN,NaN,219,893,3,3.5,...,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0.625,0.65
2,896,2013-05-09 22:14:57,5,2,NaN,NaN,219,893,3,3.5,...,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0.625,0.65
3,1495,2013-04-24 02:24:02,5,219,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.625,0.65
4,2052,2013-01-11 08:10:27,5,219,NaN,NaN,219,893,3,3.5,...,NaN,NaN,-1.0,0.0,6.0,0,NaN,0,0.625,0.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958342,332713,2013-04-15 20:51:59,12,99,NaN,NaN,99,93842,3,4.0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.000,0.70
4958343,332713,2013-04-15 20:51:59,12,99,NaN,NaN,99,103277,3,5.0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.000,0.80
4958344,332713,2013-04-15 20:51:59,12,99,NaN,NaN,99,135856,4,3.5,...,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.000,0.75
4958345,332740,2013-02-18 12:17:59,12,99,NaN,NaN,99,2284,0,0.0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.000,0.05


In [104]:
temp[temp["prop_country_id"]==1][["rank","total_score"]]

,rank,total_score
21605,0.250,3.5
36974,0.875,8.0
38545,0.625,5.5
49410,0.750,7.5
57968,0.875,8.0
66938,0.500,5.0
79621,0.125,2.0
80181,1.000,9.5
103266,0.375,4.0
112923,0.375,4.0


In [88]:
temp.groupby("prop_country_id").rank()

,prop_starrating,prop_review_score,prop_id,total_score
0,11337.0,1069.5,1.0,1366.0
1,2624.0,20137.5,2.0,5124.0
2,25800.0,12199.5,3.0,19435.5
3,2600.0,2138.0,1.0,2575.5
4,846.5,849.5,1.0,286.5
...,...,...,...,...
129108,72.0,280.0,701.0,250.5
129109,69.0,30.5,170.0,29.5
129110,360.0,503.0,799.0,510.0
129111,4719.0,830.0,6063.0,1516.5


In [ ]:
grouped_countries = temp.groupby("prop_country_id")a

In [66]:
grouped_countries = temp.groupby("prop_country_id")
temp["rank"] = grouped_countries["total_score"].rank(ascending=False)

# Step 3: Sort the DataFrame by "prop_id" and "prop_country_id" and reset the index
temp = temp.sort_values(by=["total_score"]).reset_index()
temp

,index,prop_id,prop_country_id,prop_starrating,prop_review_score,total_score,rank
0,0,9357,2,0.0,0.0,0.0,126.5
1,4081,91019,117,0.0,0.0,0.0,2799.0
2,4080,90930,117,0.0,0.0,0.0,2799.0
3,4079,90791,117,0.0,0.0,0.0,2799.0
4,4078,90589,117,0.0,0.0,0.0,2799.0
...,...,...,...,...,...,...,...
129108,129108,106989,229,2.0,NaN,NaN,NaN
129109,129109,107366,229,3.0,NaN,NaN,NaN
129110,129110,108516,229,3.0,NaN,NaN,NaN
129111,129111,127265,229,3.0,NaN,NaN,NaN


In [69]:
temp[temp["prop_country_id"]==219]

,index,prop_id,prop_country_id,prop_starrating,prop_review_score,total_score,rank
1530,5611,77956,219,0.0,0.0,0.0,37707.5
1531,5610,77820,219,0.0,0.0,0.0,37707.5
1532,5609,77630,219,0.0,0.0,0.0,37707.5
1533,5608,77517,219,0.0,0.0,0.0,37707.5
1534,5607,77462,219,0.0,0.0,0.0,37707.5
...,...,...,...,...,...,...,...
129022,129022,137610,219,0.0,NaN,NaN,NaN
129023,129023,139264,219,3.0,NaN,NaN,NaN
129024,129024,139496,219,0.0,NaN,NaN,NaN
129025,129025,140341,219,0.0,NaN,NaN,NaN
